In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

from collections import defaultdict
import pandas as pd

# Load data

In [2]:
engine = create_engine('postgresql://localhost:5432/wa_leg_raw')
con = engine.connect()

In [4]:
bill_df = pd.read_sql_query('select * from "bill_api"',con=engine)

In [5]:
topic_df = pd.read_sql_query('select * from "topic_scrape"',con=engine)

In [6]:
# vote_df = pd.read_sql_query('select * from "vote_api"',con=engine)

In [7]:
# committee_df = pd.read_sql_query('select * from "committee_api"',con=engine)

In [8]:
# committee_member_df = pd.read_sql_query('select * from "committee_member_api"',con=engine)

# Prepare topic_data

In [9]:
topic_df['bill_id'] = topic_df['bill_id'].apply(lambda x: x.split(',')[0])

In [10]:
topic_df.head()

,bill_id,bill_topic,bill_topic_expanded,year
0,HB 2682,ABANDONED PROPERTY,"Intangible property, when presumed abandoned a...",1991
1,SB 5049,ABANDONED PROPERTY,"Junk vehicles, disposal of abandoned vehicles ...",1991
2,SHB 1137,ABANDONED PROPERTY,"Uncashed checks, duties of local governments h...",1991
3,SSB 5185,ABANDONED PROPERTY,"Uncashed checks, local governments authorized ...",1991
4,SSB 5185,ABANDONED PROPERTY,"Uncashed checks, local governments authorized ...",1991


In [11]:
def change_year_to_biennium(year):
    '''Changes year to biennium. Input is string, output is string'''
    if year == '1991' or year == '1992':
        return '1991-92'
    if year == '1993' or year == '1994':
        return '1993-94'
    if year == '1995' or year == '1996':
        return '1995-96'
    if year == '1997' or year == '1998':
        return '1997-98'
    if year == '1999' or year == '2000':
        return '1999-00'
    if year == '2001' or year == '2002':
        return '2001-02'
    if year == '2003' or year == '2004':
        return '2003-04'
    if year == '2005' or year == '2006':
        return '2005-06'
    if year == '2007' or year == '2008':
        return '2007-08'
    if year == '2009' or year == '2010':
        return '2003-04'
    if year == '2011' or year == '2012':
        return '2011-12'
    if year == '2013' or year == '2014':
        return '2013-14'
    if year == '2015' or year == '2016':
        return '2015-16'
    if year == '2017' or year == '2018':
        return '2017-18'

In [12]:
topic_df['biennium'] = topic_df['year'].apply(change_year_to_biennium)

In [13]:
topic_df['bill_num'] = topic_df['bill_id'].str[-4:]

In [14]:
topic_df = topic_df.drop(['bill_topic_expanded', 'year'], axis = 1)

In [15]:
topic_df.drop_duplicates(keep='first', inplace=True)

In [50]:
topic_df[topic_df['bill_num'] == '2293']

,bill_id,bill_topic,biennium,bill_num
15,ESHB 2293,"ACCOUNTANCY, BOARD",1991-92,2293
27,ESHB 2293,ACCOUNTANTS AND ACCOUNTING,1991-92,2293
2930,ESHB 2293,COLLEGES AND UNIVERSITIES,1991-92,2293
8669,ESHB 2293,GOVERNOR,1991-92,2293
13041,ESHB 2293,MINORITIES,1991-92,2293
15279,ESHB 2293,PUBLIC FUNDS AND ACCOUNTS,1991-92,2293


In [38]:
topic_df = topic_df[topic_df['biennium'] == '1991-92']

In [44]:
t = topic_df.loc[:, ['bill_id', 'biennium', 'bill_num']]

In [45]:
t = t.drop_duplicates(keep='first')

In [46]:
len(t), len(topic_df)

(4051, 11450)

In [47]:
unique_topics = topic_df['bill_topic'].unique()
for topic in unique_topics:
    t[topic] = 0

In [48]:
for row in topic_df.iterrows():
#     print(row[1]['bill_topic'])
    t.loc[(t['bill_num'] == row[1]['bill_num']) & (t['biennium'] == row[1]['biennium']), row[1]['bill_topic']] = 1

In [28]:
t.loc[(t['bill_num'] == '1001') & (t['biennium'] == '2015-16'), 'BUDGETS']

485217    0
Name: BUDGETS, dtype: int64

In [49]:
t

,bill_id,biennium,bill_num,ABANDONED PROPERTY,ABORTION,"ACCOUNTANCY, BOARD",ACCOUNTANTS AND ACCOUNTING,ACTIONS AND PROCEEDINGS,AFRICAN-AMERICANS,ACUPUNCTURE AND ACUPUNCTURISTS,...,WOOD PRODUCTS,WOOD PRODUCTS INDUSTRY,WORK FORCE TRAINING AND EDUCATION COORDINATING BOARD,WORKERS' COMPENSATION,WRECKERS AND WRECKING YARDS,WRONGFUL DEATH ACTIONS,YAKIMA,YAKIMA COUNTY,YAKIMA VALLEY COMMUNITY COLLEGE,ZONING
0,HB 2682,1991-92,2682,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SB 5049,1991-92,5049,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SHB 1137,1991-92,1137,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SSB 5185,1991-92,5185,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,HB 2859,1991-92,2859,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,HB 1700,1991-92,1700,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,HJM 4025,1991-92,4025,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,SI 120,1991-92,120,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,ESHB 2293,1991-92,2293,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,ESHB 1588,1991-92,1588,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
bill_df['bill_num'] = bill_df['bill_id'].str[-4:]

# Make small data sets to play with

In [ ]:
topic_df_1991 = topic_df[topic_df['biennium'] == '1991-92']

In [ ]:
bill_df_1991 = bill_df[bill_df['biennium'] == '1991-92']

In [ ]:
merged_df = bill_df.merge(topic_df, how='outer', on=['bill_num', 'biennium'])

In [ ]:
len(merged_df)

In [ ]:
len(merged_df[merged_df['bill_topic'].isnull()])

In [ ]:
len(merged_df['bill_topic'].unique())

In [ ]:
merged_df['bill_topic']

In [ ]:
x = ['1', '2', '3', '1']
y = ['a', 'b', 'a', 'a']
z = ['ww', 'xx', 'yy', 'zz']
dct1 = defaultdict(list)
for x, y, z in zip(x, y, z):
    dct1[(x, y)].append(z)

In [ ]:
dct1

In [ ]:
merged_df